In [38]:
import os
import glob
import polars as pl
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from utils import hermes_download_client
from utils import ambient_parameter_conversion as apc
from utils import calibration_processing as cp

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")
PICARRO_DATA_DIRECTORY = os.environ.get("PICARRO_DATA_DIRECTORY")

sensor_id = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,19,20]

# customize pipeline
download_files = True
merge_picarro_files = False

# Download to local db

In [39]:
# download from hermes database
# Use Download/download_from_hermes notebook
if download_files:
    
    while(True):
        try:
            component = hermes_download_client.Extract()
            result = component.execute()
            break
        except:
            print("restart download. error while excution.")

Start downloading from datetime: 
2024-03-25 12:07:51.982378+00:00
2024-03-25 12:25:18.140261+00:00
2024-03-25 12:33:37.500026+00:00
2024-03-25 12:49:04.854517+00:00
2024-03-25 13:04:29.902747+00:00
2024-03-25 13:13:49.011535+00:00
2024-03-25 13:28:22.173844+00:00
2024-03-25 13:42:58.277547+00:00
2024-03-25 13:57:32.736583+00:00
Performing merge.
Deleting merged chunks.


# Process Picarro Data

In [40]:
if merge_picarro_files:
    filenames = glob.glob(PICARRO_DATA_DIRECTORY + "/*/*/*.dat")

    # read all *.dat picarro measurement files and add to single db
    df_list = []
    for filename in filenames:
        df_list.append(pd.read_csv(filename,sep='\s+'))

    df_p_files = pd.concat(df_list, ignore_index=True)
    df_p_files["datetime"] = pd.to_datetime((df_p_files['DATE'] + ' ' + df_p_files['TIME']))
    df_p_files.sort_values(by='datetime', inplace = True)

    df_p_files.to_parquet(path = os.path.join(DATA_DIRECTORY, "input", "picarro.parquet"))

    #Calibration

    # TODO: Add ability for multiple calibration dates
    # before 23.10
    # picarro_slope = 1.0061589132696314
    # picarro_intercept = 0.14607153970888476

    # after 23.10
    #picarro_slope = 1.0063874771746113
    #picarro_intercept = 0.06621464961165202
    
     
    #after 18.12
    picarro_slope = 1.0060713120261249
    picarro_intercept = 0.08088569875155827
    
    #10m averaged corrected Picarro dataset
    df_p_10m = pl.scan_parquet(os.path.join(DATA_DIRECTORY,"input", "picarro.parquet")) \
        .with_columns(pl.col("datetime").dt.cast_time_unit("us").dt.replace_time_zone("UTC").alias("creation_timestamp")) \
        .sort("creation_timestamp") \
        .with_columns((pl.col("CO2_dry") * picarro_slope + picarro_intercept).alias("CO2_corr")) \
        .group_by_dynamic("creation_timestamp", every='10m') \
        .agg(pl.all().exclude("creation_timestamp").mean()).collect() \
        .select(["creation_timestamp", "CO2_corr", "h2o_reported"]) \
        .with_columns([pl.lit(picarro_slope).alias("slope"),
            pl.lit(picarro_intercept).alias("intercept"),
            pl.lit("Picarro").alias("system_name"),
            pl.lit(0.0).alias("diff")]
            ) \

    df_p_10m.write_parquet(os.path.join(DATA_DIRECTORY,"processed", "10m_cal_corr_picarro.parquet"))
    
    #1h averaged corrected Picarro dataset
    df_p_1h = df_p_10m.sort("creation_timestamp") \
        .group_by_dynamic("creation_timestamp", every='1h') \
        .agg(pl.all().exclude("creation_timestamp").mean())
        
    df_p_1h.write_parquet(os.path.join(DATA_DIRECTORY,"processed", "1h_cal_corr_picarro.parquet"))

# Perform Dry-Wet Conversion

# Import data files

In [41]:
# load calibration bottle concentrations (preprocessed)
df_gas = pl.read_csv(os.path.join(DATA_DIRECTORY,"input", "averaged_gases.csv"))
# load local db: acropolis.parquet
df_raw = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "download", "acropolis.parquet")) \
    .filter(pl.col("system_name") != "test-sensor") 
    
df_p_10m = pl.read_parquet(os.path.join(DATA_DIRECTORY,"processed", "10m_cal_corr_picarro.parquet"))  

### Measurement Data

In [42]:
# aggregate data to 10m
df_wet_measurements = df_raw.sort("creation_timestamp") \
    .filter(pl.col('gmp343_filtered') > 0) \
    .filter(pl.col('gmp343_temperature') > 0) \
    .filter(pl.col('sht45_humidity') > 0) \
    .filter(pl.col('bme280_pressure') > 0) \
    .group_by_dynamic("creation_timestamp", every='10m', by= "system_name")  \
    .agg(pl.all().exclude(["creation_timestamp","system_name"]).mean()) \
    
# perform dry conversion for measurement data                
df_dry_measurements = df_wet_measurements.with_columns(pl.struct(['gmp343_temperature','sht45_humidity']) \
    .map_elements(lambda x: apc.rh_to_ah(x['sht45_humidity'],apc.absolute_temperature(x['gmp343_temperature'])))
    .alias("h2o_ah")) \
    .with_columns(pl.struct(['gmp343_temperature','sht45_humidity','bme280_pressure'])
    .map_elements(lambda x: (apc.rh_to_molar_mixing(x['sht45_humidity'],apc.absolute_temperature(x['gmp343_temperature']),x['bme280_pressure']*100))*100) \
    .alias("h2o_v%")) \
    .with_columns(pl.struct(['gmp343_filtered','gmp343_temperature','sht45_humidity','bme280_pressure']) \
    .map_elements(lambda x: apc.calculate_co2dry(x['gmp343_filtered'],x['gmp343_temperature'],x['sht45_humidity'],x['bme280_pressure']*100))
    .alias("gmp343_dry")) \
    .collect() 

In [43]:
df_dry_measurements.tail(3).select("creation_timestamp","system_name","gmp343_filtered", "h2o_ah", "h2o_v%" ,"gmp343_dry")

creation_timestamp,system_name,gmp343_filtered,h2o_ah,h2o_v%,gmp343_dry
"datetime[μs, UTC]",str,f64,f64,f64,f64
2024-03-25 11:00:00 UTC,"""tum-esm-midcos…",436.143333,6.833525,1.014315,440.61253
2024-03-25 11:10:00 UTC,"""tum-esm-midcos…",435.276667,6.425677,0.954225,439.470203
2024-03-25 11:20:00 UTC,"""tum-esm-midcos…",436.556522,6.007084,0.892196,440.486523


In [44]:
df_dry_measurements.write_parquet(os.path.join(DATA_DIRECTORY, "processed", "acropolis_dry.parquet"))

### Calibration Data

In [45]:
df_dry_calibration = df_raw.filter(pl.col("cal_gmp343_filtered") > 0) \
    .filter(pl.col("cal_gmp343_temperature") > 0) \
    .filter(pl.col("cal_bme280_pressure") > 0) \
    .with_columns(pl.col("cal_sht45_humidity").fill_null(0.0)) \
    .with_columns(pl.struct(['cal_gmp343_temperature','cal_sht45_humidity','cal_bme280_pressure'])
    .map_elements(lambda x: apc.rh_to_molar_mixing(x['cal_sht45_humidity'],apc.absolute_temperature(x['cal_gmp343_temperature']),x['cal_bme280_pressure']*100)) \
    .alias("cal_h2o_v%")) \
    .with_columns(pl.struct(['cal_gmp343_filtered','cal_gmp343_temperature','cal_sht45_humidity','cal_bme280_pressure']) \
    .map_elements(lambda x: apc.calculate_co2dry(x['cal_gmp343_filtered'],x['cal_gmp343_temperature'],x['cal_sht45_humidity'],x['cal_bme280_pressure']*100))
    .alias("cal_gmp343_dry")) \
    .collect()

In [46]:
df_dry_calibration.tail(3).select("creation_timestamp","system_name","cal_gmp343_filtered", "cal_h2o_v%", "cal_gmp343_dry")

creation_timestamp,system_name,cal_gmp343_filtered,cal_h2o_v%,cal_gmp343_dry
"datetime[μs, UTC]",str,f64,f64,f64
2024-03-25 03:43:54.230 UTC,"""tum-esm-midcos…",419.6,0.000968,420.006638
2024-03-25 03:44:04.190 UTC,"""tum-esm-midcos…",421.5,0.000971,421.90961
2024-03-25 03:44:14.200 UTC,"""tum-esm-midcos…",423.7,0.00098,424.115832


In [47]:
df_slope_intercept = df_dry_calibration.filter((pl.col("cal_bottle_id") > 0) & (pl.col("cal_bottle_id") < 81)) \
    .join(df_gas.cast({"cal_bottle_id": pl.Float64}), on = ["cal_bottle_id"], how= "left") \
    .with_columns(diff = pl.col("cal_gmp343_dry") - pl.col("cal_bottle_CO2")) \
    .with_columns((pl.col("creation_timestamp").dt.date()).alias("date")) \
    .sort("date") \
    .group_by([pl.col("date"), pl.col("system_name"), pl.col("cal_bottle_id")]) \
    .agg([
        pl.col("cal_sht45_humidity"),
        pl.col("cal_gmp343_temperature"),
        pl.col("cal_bme280_pressure"),
        pl.col("cal_gmp343_filtered"),
        pl.col("cal_gmp343_dry"),
        pl.col("cal_bottle_CO2").last(),
        pl.col("creation_timestamp").last(),
        ]) \
    .with_columns([
        (pl.col("cal_sht45_humidity").list.len()).alias("len"),
        pl.when(pl.col("cal_bottle_CO2") > 460).then(pl.lit("high")).otherwise(pl.lit("low")).alias("cal_bottle_span"),
        pl.col("cal_gmp343_dry").map_elements(lambda x: cp.process_bottle(x)).alias("median_cal_gmp343_dry"),
        pl.col("cal_gmp343_temperature").map_elements(lambda x: cp.process_bottle(x)).alias("median_cal_gmp343_temperature"),
        pl.col("cal_bme280_pressure").map_elements(lambda x: cp.process_bottle(x)).alias("median_cal_bme280_pressure"),
        pl.col("cal_sht45_humidity").map_elements(lambda x: cp.process_bottle(x)).alias("median_cal_sht45_humidity")
        ]) \
    .with_columns([
        (pl.col("median_cal_gmp343_dry") - pl.col("cal_bottle_CO2")).alias("diff_cut_avg_gmp343_dry")
                  ]) \
    .filter(pl.col("median_cal_gmp343_dry") > 0) \
    .sort("cal_bottle_span") \
    .group_by(["date", "system_name"]) \
    .agg([
        pl.col("cal_bottle_span"),
        pl.col("median_cal_gmp343_dry"),
        pl.col("cal_bottle_CO2"),
        pl.col("creation_timestamp").last()
        ]) \
    .with_columns(pl.struct(['median_cal_gmp343_dry','cal_bottle_CO2']) \
    .map_elements(lambda x: cp.two_point_calibration(x['median_cal_gmp343_dry'],x['cal_bottle_CO2'])) \
    .alias('slope, intercept')) \
    .with_columns([(pl.col("slope, intercept").list.first()).alias("slope"),
                   (pl.col("slope, intercept").list.last()).alias("intercept"),
                   (pl.col("median_cal_gmp343_dry").list.first()).alias("median_bottle_1"),
                   (pl.col("median_cal_gmp343_dry").list.last()).alias("median_bottle_2"),
                   ]) \
    .select("creation_timestamp", "date", "system_name", "slope", "intercept","median_bottle_1", "median_bottle_2") \
    .filter(pl.col("slope") > 0) \
    .with_columns(pl.col("system_name").str.extract(r'(\d+)',1).str.to_integer().alias("system_id"))

In [48]:
df_slope_intercept.head(3)

creation_timestamp,date,system_name,slope,intercept,median_bottle_1,median_bottle_2,system_id
"datetime[μs, UTC]",date,str,i64,i64,f64,f64,i64
2023-07-03 05:32:11 UTC,2023-07-03,"""tum-esm-midcos…",1,-8,791.55,391.014313,2
2023-07-08 10:21:37.220 UTC,2023-07-08,"""tum-esm-midcos…",1,-9,791.6,402.15,7
2023-07-10 12:20:56.010 UTC,2023-07-10,"""tum-esm-midcos…",1,-13,788.6,402.5,7


In [49]:
# safe results to parquet
df_slope_intercept.write_parquet(os.path.join(DATA_DIRECTORY, "processed", "slope_intercept_acropolis.parquet"))

In [50]:
fig = px.scatter(df_slope_intercept.filter((pl.col("slope") > 0.7) & (pl.col("slope") < 1.1)), x="creation_timestamp", y = "slope", color = "system_name")
fig.show()
fig = px.histogram(df_slope_intercept.filter((pl.col("slope") > 0.7) & (pl.col("slope") < 1.1)), x="slope", color = "system_name")
fig.show()
fig = px.histogram(df_slope_intercept.filter((pl.col("intercept") < 100) & (pl.col("intercept") > -100)), x="intercept", color = "system_name")
fig.show()

# Perform Calibration Correction

## 10m aggregated data

In [51]:
df_dry_measurements = df_dry_measurements.with_columns(pl.col("system_name").str.extract(r'(\d+)',1) \
    .str.to_integer().alias("system_id")) \
    .with_columns(pl.struct(["system_id"]) \
    .map_elements(lambda x: f"mid-cost-{x['system_id']}") \
    .alias("sys_name_short"))

In [52]:
df_systems = []

for id in sensor_id:
    df_system_calibration = df_slope_intercept.filter(pl.col("system_id") == id) \
        .sort("creation_timestamp") \
        .drop("system_id", "system_name", "date")
    
    df_system = df_dry_measurements.filter(pl.col("system_id") == id) \
        .sort("creation_timestamp") \
        .join_asof(df_system_calibration, on="creation_timestamp", strategy="nearest", tolerance="10m") \
        .join(df_p_10m.select("creation_timestamp", "CO2_corr"), on = ["creation_timestamp"], how= "left") \
        .with_columns([
            pl.col("slope").interpolate(),
            pl.col("intercept").interpolate()
            ]) \
        .with_columns([
            pl.col("slope").forward_fill(),
            pl.col("intercept").forward_fill()
            ]) \
        .with_columns(((pl.col("gmp343_dry")) * pl.col("slope") + pl.col("intercept")).alias("gmp343_corrected")) \
        .with_columns((pl.col("CO2_corr") - pl.col("gmp343_corrected")).alias("diff")) \
        .with_columns((pl.col("creation_timestamp").dt.date()).alias("date"))
    
    df_systems.append(df_system)
        

df_dry_corrected_10m = pl.concat(df_systems, how="diagonal")

In [53]:
df_dry_corrected_10m.write_parquet(os.path.join(DATA_DIRECTORY, "processed", "10m_cal_corr_acropolis.parquet"))

In [54]:
# Quality Check

from datetime import datetime, timezone

start = datetime(2024, 1, 13, 0, 0, 0).replace(tzinfo=timezone.utc)
end = datetime(2024, 2, 18, 23, 59, 59).replace(tzinfo=timezone.utc)

df_temp = df_dry_corrected_10m.filter(pl.col("system_id") == 3).filter(pl.col("creation_timestamp").is_between(start, end))

print(df_temp.select("diff").mean())

fig = px.scatter(df_temp, x="creation_timestamp", y = "diff", color = "system_name")
fig.show()

shape: (1, 1)
┌────────────┐
│ diff       │
│ ---        │
│ f64        │
╞════════════╡
│ -22.597025 │
└────────────┘


In [55]:
df_dry_corrected_1h = df_dry_corrected_10m.sort("creation_timestamp") \
        .group_by_dynamic("creation_timestamp", every='1h', by=["system_name", "sys_name_short"]) \
        .agg(pl.all().exclude(["creation_timestamp","system_name","sys_name_short"]).mean())
        
df_dry_corrected_1h.write_parquet(os.path.join(DATA_DIRECTORY, "processed", "1h_cal_corr_acropolis.parquet"))